# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

¡Empecemos!

In [ ]:
url = 'https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=20&fecha1_anomes=2019-12&auAvailability=1&ida_vuelta=ida&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=20/12/2019&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo='

In [ ]:
from selenium import webdriver

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='../../chromedriver', options=options)
driver.get(url)

In [ ]:
#Usaremos el Xpath para obtener la lista de vuelos
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')

In [ ]:
vuelo = vuelos[0]

Obtenemos la información de la hora de salida, llegada y duración del vuelo

In [ ]:
# Hora de salida
vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

In [ ]:
# Hora de llegada
vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

In [ ]:
# Duración del vuelo
vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

In [ ]:
boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')
boton_escalas

In [ ]:
boton_escalas.click()

In [ ]:
segmentos = vuelo.find_elements_by_xpath('//div[@class="segments-graph"]/div[@class="segments-graph-segment"]')
segmentos

In [ ]:
escalas = len(segmentos) - 1 #0 escalas si es un vuelo directo

In [ ]:
segmento = segmentos[0]

In [ ]:
# Origen
segmento.find_element_by_xpath('.//div[@class="departure"]/span[@class="ground-point-name"]').text

In [ ]:
# Hora de salida
segmento.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

In [ ]:
# Destino
segmento.find_element_by_xpath('.//div[@class="arrival"]/span[@class="ground-point-name"]').text

In [ ]:
# Hora de llegada
segmento.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

In [ ]:
# Duración del vuelo
segmento.find_element_by_xpath('.//span[@class="duration flight-schedule-duration"]/time').get_attribute('datetime')

In [ ]:
# Numero del vuelo
segmento.find_element_by_xpath('.//span[@class="equipment-airline-number"]').text

In [ ]:
# Modelo de avion
segmento.find_element_by_xpath('.//span[@class="equipment-airline-material"]').text

In [ ]:
# Duracion de la escala
segmento.find_element_by_xpath('.//div[@class="stop connection"]//p[@class="stop-wait-time"]//time').get_attribute('datetime')

In [ ]:
vuelo.find_element_by_xpath('//div[@class="modal-dialog"]//button[@class="close"]').click()

In [ ]:
vuelo.click()

In [ ]:
tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')

In [ ]:
precios = []
for tarifa in tarifas:
    nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
    moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
    valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text 
    dict_tarifa={nombre:{'moneda':moneda, 'valor':valor}}
    precios.append(dict_tarifa)
    print(dict_tarifa)

In [ ]:
def obtener_tiempos(vuelo):
    # Hora de salida
    salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
    # Hora de llegada
    llegada = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
    # Duracion
    duracion = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
    return {'hora_salida': salida, 'hora_llegada': llegada, 'duracion': duracion}

In [ ]:
def obtener_precios(vuelo):
    tarifas = vuelo.find_elements_by_xpath(
        './/div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')
    precios = []
    for tarifa in tarifas:
        nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
        moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
        valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text 
        dict_tarifa={nombre:{'moneda':moneda, 'valor':valor}}
        precios.append(dict_tarifa)
    return precios

In [ ]:
def obtener_datos_escalas(vuelo):
    segmentos = vuelo.find_elements_by_xpath('//div[@class="segments-graph"]/div[@class="segments-graph-segment"]')
    info_escalas = []
    for segmento in segmentos:
        # Origen
        origen = segmento.find_element_by_xpath(
            './/div[@class="departure"]/span[@class="ground-point-name"]').text
        # Hora de salida
        dep_time = segmento.find_element_by_xpath(
            './/div[@class="departure"]/time').get_attribute('datetime')
        # Destino
        destino = segmento.find_element_by_xpath(
            './/div[@class="arrival"]/span[@class="ground-point-name"]').text
        # Hora de llegada
        arr_time = segmento.find_element_by_xpath(
            './/div[@class="arrival"]/time').get_attribute('datetime')
        # Duración del vuelo
        duracion_vuelo = segmento.find_element_by_xpath(
            './/span[@class="duration flight-schedule-duration"]/time').get_attribute('datetime')
        # Numero del vuelo
        numero_vuelo = segmento.find_element_by_xpath(
            './/span[@class="equipment-airline-number"]').text
        # Modelo de avion
        modelo_avion = segmento.find_element_by_xpath(
            './/span[@class="equipment-airline-material"]').text
        # Duracion de la escala
        if segmento != segmentos[-1]:
            duracion_escala = segmento.find_element_by_xpath(
                './/div[@class="stop connection"]//p[@class="stop-wait-time"]//time').get_attribute('datetime')
        else:
            duracion_escala = ''

        # Armo un diccionario para almacenar los datos
        data_dict={'origen': origen, 
                    'dep_time': dep_time, 
                    'destino': destino,
                    'arr_time': arr_time,
                    'duracion_vuelo': duracion_vuelo,
                    'numero_vuelo': numero_vuelo,
                    'modelo_avion': modelo_avion,
                    'duracion_escala': duracion_escala}
        info_escalas.append(data_dict)
        
    return info_escalas

## Clase 5
Ya tenemos el scraper casi listo. Unifiquemos las 3 funciones de la clase anterior en una sola